In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.classification import GBTClassifier, GBTClassificationModel
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pandas as pd
from pyspark.ml import Pipeline,Transformer, PipelineModel
from pyspark.sql import DataFrame
from typing import Iterable
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import LinearRegressionSummary
from pyspark.ml.classification import LinearSVC, LinearSVCModel
from pyspark.ml.stat import Correlation
from pyspark.ml.util import MLReadable, MLWritable
#import mlflow
from pyspark.conf import SparkConf

In [2]:
sqlContext.setConf("spark.sql.shuffle.partitions", "2")
sqlContext.setConf("spark.default.parallelism", "2")
print(sqlContext.getConf("spark.default.parallelism"))
print(sqlContext.getConf("spark.sql.shuffle.partitions"))

2
2

In [3]:
%fs
ls /FileStore/olist/ENTREGA_2/0_bases/

path,name,size
dbfs:/FileStore/olist/ENTREGA_2/0_bases/ABT_com_nulo.parquet/,ABT_com_nulo.parquet/,0
dbfs:/FileStore/olist/ENTREGA_2/0_bases/ABT_com_nulo_e_unique_id.parquet/,ABT_com_nulo_e_unique_id.parquet/,0
dbfs:/FileStore/olist/ENTREGA_2/0_bases/ABT_nulos_replace_zero_entrega2.parquet/,ABT_nulos_replace_zero_entrega2.parquet/,0
dbfs:/FileStore/olist/ENTREGA_2/0_bases/DataFrame_completo_para_modelo.parquet/,DataFrame_completo_para_modelo.parquet/,0
dbfs:/FileStore/olist/ENTREGA_2/0_bases/teste_entrega2.parquet/,teste_entrega2.parquet/,0
dbfs:/FileStore/olist/ENTREGA_2/0_bases/treino_entrega2.parquet/,treino_entrega2.parquet/,0


#5.7 Maximizar os Hiperparametros

##5.7.1 ParamGridBuilder
Construir uma rede de parâmetros otimizada para obter o melhor modelo

####LogisticRegressionModel 
https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegression

In [7]:
"""
LogisticRegression(self, 
                   featuresCol="features", 
                   labelCol="label", 
                   predictionCol="prediction", 
                   maxIter=100, 
                   regParam=0.0, 
                   elasticNetParam=0.0, 
                   tol=1e-6, 
                   fitIntercept=True, 
                   threshold=0.5, 
                   thresholds=None, 
                   probabilityCol="probability", 
                   rawPredictionCol="rawPrediction", 
                   standardization=True, 
                   weightCol=None, 
                   aggregationDepth=2, 
                   family="auto")
                   
 - elasticNetParam = Param(parent='undefined', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. 
                     For alpha = 0, the penalty is an L2 penalty. 
                     For alpha = 1, it is an L1 penalty.')
 - family = Param(parent='undefined', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: 
                     auto, 
                     binomial, 
                     multinomial')
 - maxIter = Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0))
 - regParam = Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).')
 - threshold = Param(parent='undefined', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].')
 - thresholds = Param(parent='undefined', name='thresholds', doc="Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold.")
 - tol = Param(parent='undefined', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).')
 - aggregationDepth = Param(parent='undefined', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).')
 - standardization = Param(parent='undefined', name='standardization', doc='whether to standardize the training features before fitting the model.')
"""
#modelo_reg_log_entrega2
new_LogisticRegression = LogisticRegression(labelCol='target')
paramGrid_LogisticRegressionModel = (ParamGridBuilder()
                                     .addGrid(new_LogisticRegression.maxIter, [10, 50, 100])
                                     .addGrid(new_LogisticRegression.regParam, [0.3, 0.05, 0.001])
                                     .addGrid(new_LogisticRegression.elasticNetParam, [0.0, 0.8])
                                     .build())


####DecisionTree
https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html#pyspark.ml.classification.DecisionTreeClassifier

In [9]:
"""
DecisionTreeClassifier(self, 
                       featuresCol="features", 
                       labelCol="label", 
                       predictionCol="prediction", 
                       probabilityCol="probability", 
                       rawPredictionCol="rawPrediction", 
                       maxDepth=5, 
                       maxBins=32, 
                       minInstancesPerNode=1, 
                       minInfoGain=0.0, 
                       maxMemoryInMB=256, 
                       cacheNodeIds=False, 
                       checkpointInterval=10, 
                       impurity="gini", 
                       seed=None)
                       
- impurity = Param(parent='undefined', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: 
              entropy, 
              gini')
- maxBins = Param(parent='undefined', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.')
- 
"""

# DecisionTree_entrega2
new_DecisionTreeClassifier = DecisionTreeClassifier(featuresCol='features', labelCol = 'target')
paramGrid_DecisionTree = (ParamGridBuilder()
                          .addGrid(new_DecisionTreeClassifier.maxDepth, [10, 30])
                          .addGrid(new_DecisionTreeClassifier.maxBins, [10, 50])
                          .addGrid(new_DecisionTreeClassifier.minInfoGain, [0.01, 0.8])
                          .addGrid(new_DecisionTreeClassifier.impurity, ['entropy'])
                          .addGrid(new_DecisionTreeClassifier.seed, [42])
                          .build())

####SVM
https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html#pyspark.ml.classification.LinearSVC

In [11]:
"""
LinearSVC(featuresCol='features',  
          labelCol='label', 
          predictionCol='prediction', 
          maxIter=100, 
          regParam=0.0, 
          tol=1e-06, 
          rawPredictionCol='rawPrediction', 
          fitIntercept=True, 
          standardization=True, 
          threshold=0.0, 
          weightCol=None, 
          aggregationDepth=2)
"""


#modelo_svm
# #new_LinearSVC = LinearSVC(featuresCol='features', labelCol = 'target')
# #paramGrid_LinearSVC = (ParamGridBuilder()
# #                       .addGrid(new_LinearSVC.maxIter, [10, 50, 100])
# #                       .addGrid(new_LinearSVC.regParam, [0.3, 0.1, 0.01, 0.05, 0.001])
# #                       .build())
new_LinearSVC2 = LinearSVC(featuresCol='features', labelCol = 'target')
paramGrid_LinearSVC2 = (ParamGridBuilder()
                       .addGrid(new_LinearSVC2.maxIter, [100, 500])
                       .addGrid(new_LinearSVC2.regParam, [0.01, 0.001])
                       .build())

####RandomForestClassifier
https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html#pyspark.ml.classification.RandomForestClassifier

In [13]:
"""
RandomForestClassifier(self, 
                       featuresCol="features", 
                       labelCol="label", 
                       predictionCol="prediction", 
                       probabilityCol="probability", 
                       rawPredictionCol="rawPrediction", 
                       maxDepth=5, 
                       maxBins=32, 
                       minInstancesPerNode=1, 
                       minInfoGain=0.0, 
                       maxMemoryInMB=256, 
                       cacheNodeIds=False, 
                       checkpointInterval=10, 
                       impurity="gini", 
                       numTrees=20, 
                       featureSubsetStrategy="auto", 
                       seed=None, 
                       subsamplingRate=1.0)
                     
- cacheNodeIds = Param(parent='undefined', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.')
- checkpointInterval = Param(parent='undefined', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.')
- featureSubsetStrategy = Param(parent='undefined', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 
                          'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 
                          'all' (use all features), 
                          'onethird' (use 1/3 of the features), 
                          'sqrt' (use sqrt(number of features)), 
                          'log2' (use log2(number of features)), 
                          'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). 
                          default = 'auto'")
- impurity = Param(parent='undefined', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: 
                          entropy, 
                          gini')
- maxBins = Param(parent='undefined', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.')
- maxDepth = Param(parent='undefined', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.')
- maxMemoryInMB = Param(parent='undefined', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.')
- minInfoGain = Param(parent='undefined', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.')
- minInstancesPerNode = Param(parent='undefined', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.')
- numTrees = Param(parent='undefined', name='numTrees', doc='Number of trees to train (>= 1).')
- subsamplingRate = Param(parent='undefined', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].')
"""

# modelo_rf
RandomForestClassifier1 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 3, maxBins= 10, numTrees= 10, impurity='entropy')
# RandomForestClassifier2 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 3, maxBins= 40, numTrees= 10, impurity='entropy')
# RandomForestClassifier3 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 3, maxBins= 50, numTrees= 10, impurity='entropy')
# RandomForestClassifier4 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 3, maxBins= 10, numTrees= 50, impurity='entropy')
# RandomForestClassifier5 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 3, maxBins= 40, numTrees= 50, impurity='entropy')
# RandomForestClassifier6 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 3, maxBins= 50, numTrees= 50, impurity='entropy')
# RandomForestClassifier7 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 20, maxBins= 10, numTrees= 10, impurity='entropy')
# RandomForestClassifier8 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 20, maxBins= 40, numTrees= 10, impurity='entropy')
# RandomForestClassifier9 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 20, maxBins= 50, numTrees= 10, impurity='entropy')
# RandomForestClassifier10 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 20, maxBins= 10, numTrees= 50, impurity='entropy')
# RandomForestClassifier11 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 20, maxBins= 40, numTrees= 50, impurity='entropy')

RandomForestClassifier12 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 20, maxBins= 50, numTrees= 50, impurity='entropy')
# RandomForestClassifier13 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 30, maxBins= 70, numTrees= 70, impurity='entropy')
# RandomForestClassifier14 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 30, maxBins= 70, numTrees= 100, impurity='entropy')
RandomForestClassifier15 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 30, maxBins= 100, numTrees= 70, impurity='entropy')
RandomForestClassifier16 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 30, maxBins= 100, numTrees= 100, impurity='entropy')

RandomForestClassifier17 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 30, maxBins= 100, numTrees= 100, impurity='entropy',minInfoGain=0.01)
RandomForestClassifier18 = RandomForestClassifier(featuresCol='features', labelCol = 'target', maxDepth= 30, maxBins= 100, numTrees= 100, impurity='entropy',minInfoGain=0.8)
# paramGrid_RandomForestClassifier = (ParamGridBuilder()
#                                     .addGrid(new_RandomForestClassifier.maxDepth, [5, 30])
#                                     .addGrid(new_RandomForestClassifier.maxBins, [100])
#                                     #.addGrid(new_RandomForestClassifier.minInfoGain, [0.01, 0.8])
#                                     #.addGrid(new_RandomForestClassifier.impurity, ['entropy', 'gini'])
#                                     .addGrid(new_RandomForestClassifier.numTrees, [10, 50])
#                                     .build())

####Gradient-boosted tree classifier
https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html#pyspark.ml.classification.GBTClassifier

In [15]:
"""
class pyspark.ml.classification.GBTClassifier(featuresCol='features', 
                                              labelCol='label', 
                                              predictionCol='prediction',
                                              maxDepth=5, 
                                              maxBins=32,
                                              minInstancesPerNode=1,
                                              minInfoGain=0.0,
                                              maxMemoryInMB=256, 
                                              cacheNodeIds=False, 
                                              checkpointInterval=10, 
                                              lossType='logistic', 
                                              maxIter=20, 
                                              stepSize=0.1, 
                                              seed=None, 
                                              subsamplingRate=1.0, 
                                              featureSubsetStrategy='all')
- cacheNodeIds = Param(parent='undefined', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.')
- checkpointInterval = Param(parent='undefined', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.')
- featureSubsetStrategy = Param(parent='undefined', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 
                          'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 
                          'all' (use all features), 
                          'onethird' (use 1/3 of the features), 
                          'sqrt' (use sqrt(number of features)), 
                          'log2' (use log2(number of features)), 
                          'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). 
                          default = 'auto'")
- lossType = Param(parent='undefined', name='lossType', doc='Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic')
- maxBins = Param(parent='undefined', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.')
- maxDepth = Param(parent='undefined', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.')
- maxIter = Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).')
- maxMemoryInMB = Param(parent='undefined', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.')
- minInfoGain = Param(parent='undefined', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.')
- minInstancesPerNode = Param(parent='undefined', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.')
- stepSize = Param(parent='undefined', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator.')
- subsamplingRate = Param(parent='undefined', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].')
"""

# modelo_gbt
# new_GBTClassifier1 = GBTClassifier(featuresCol='features', labelCol = 'target')
# paramGrid_GBTClassifier1 = (ParamGridBuilder()
#                            .addGrid(new_GBTClassifier1.maxDepth, [3, 10, 30])
#                            .addGrid(new_GBTClassifier1.maxBins, [10, 20])
#                            .addGrid(new_GBTClassifier1.minInfoGain, [0.01,0.3])
#                            .addGrid(new_GBTClassifier1.maxIter, [10, 100])
#                            .build())

#como não funcionou no crosvalidator, então fizemos manualmente 
new_GBTClassifier1 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 10, minInfoGain = 0.01)
# new_GBTClassifier2 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 10, minInfoGain = 0.3)
# new_GBTClassifier3 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 20, minInfoGain = 0.01)
# new_GBTClassifier4 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 20, minInfoGain = 0.3)

# new_GBTClassifier5 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 10, minInfoGain = 0.01)
# new_GBTClassifier6 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 10, minInfoGain = 0.3)
# new_GBTClassifier7 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 20, minInfoGain = 0.01)
# new_GBTClassifier8 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 20, minInfoGain = 0.3)

# new_GBTClassifier9 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 10, minInfoGain = 0.01)
# new_GBTClassifier10 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 10, minInfoGain = 0.3)
# new_GBTClassifier11 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 20, minInfoGain = 0.01)
# new_GBTClassifier12 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 20, minInfoGain = 0.3)

# new_GBTClassifier13 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 10, minInfoGain = 0.01, maxIter = 10)
# new_GBTClassifier14 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 10, minInfoGain = 0.3, maxIter = 10)
new_GBTClassifier15 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 20, minInfoGain = 0.01, maxIter = 10)
# new_GBTClassifier16 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 20, minInfoGain = 0.3, maxIter = 10)

# new_GBTClassifier17 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 10, minInfoGain = 0.01, maxIter = 10)
# new_GBTClassifier18 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 10, minInfoGain = 0.3, maxIter = 10)
# new_GBTClassifier19 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 20, minInfoGain = 0.01, maxIter = 10)
# new_GBTClassifier20 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 20, minInfoGain = 0.3, maxIter = 10)

# new_GBTClassifier21 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 10, minInfoGain = 0.01, maxIter = 10)
# new_GBTClassifier22 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 10, minInfoGain = 0.3, maxIter = 10)
# new_GBTClassifier23 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 20, minInfoGain = 0.01, maxIter = 10)
# new_GBTClassifier24 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 20, minInfoGain = 0.3, maxIter = 10)

# new_GBTClassifier25 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 10, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier26 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 10, minInfoGain = 0.3, maxIter = 100)
# new_GBTClassifier27 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 20, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier28 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 3, maxBins = 20, minInfoGain = 0.3, maxIter = 100)

# new_GBTClassifier29 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 10, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier30 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 10, minInfoGain = 0.3, maxIter = 100)
# new_GBTClassifier31 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 20, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier32 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 10, maxBins = 20, minInfoGain = 0.3, maxIter = 100)

# new_GBTClassifier33 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 10, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier34 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 10, minInfoGain = 0.3, maxIter = 100)
new_GBTClassifier35 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 20, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier36 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 20, minInfoGain = 0.3, maxIter = 100)

new_GBTClassifier37 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 50, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier38 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 60, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier39 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 80, minInfoGain = 0.01, maxIter = 100)
# new_GBTClassifier40 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 50, minInfoGain = 0.001, maxIter = 100)
# new_GBTClassifier41 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 60, minInfoGain = 0.001, maxIter = 100)
# new_GBTClassifier42 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 80, minInfoGain = 0.001, maxIter = 100)
new_GBTClassifier43 = GBTClassifier(featuresCol='features', labelCol = 'target', maxDepth = 30, maxBins = 100, minInfoGain = 0.001, maxIter = 100)

##5.7.2 CrossValidator
https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator

A validação cruzada executa a seleção do modelo dividindo o conjunto de dados em um conjuntos aleatórios não sobrepostos, usadas como conjuntos de dados de treinamento e teste separados, por exemplo, com k = 3 , a validação cruzada gera 3 (treinamento, pares de conjuntos de dados de teste), cada um dos quais usa 2/3 dos dados para treinamento e 1/3 para teste. Cada conjunto é usado como o conjunto de testes exatamente uma vez.

In [17]:
class vetorizar(Transformer):
    """
    Faz o Vector assembler baseado numa base recebida, com isso conseguimos usar no pipeline
    """
    def __init__(self):
        super(vetorizar, self).__init__()
        self.DataFrame_resultante = None
        
    def transform(self, base,verbose=False):
        if verbose:
          print("*****  Vetorizando a base  *****")
        assembler = VectorAssembler( inputCols = base.columns[:-1] , outputCol = 'features')
        self.DataFrame_resultante = assembler.transform(base)
        if verbose:
          print("*****  base vetorizada, pronta para rodar o modelo  *****")
        return self.DataFrame_resultante

In [18]:
def metricas(modelo,base):
  """
  Função para retornar as metricas de avaliação do modelo.
  """
  predicao = modelo.transform(base)
  #MulticlassClassificationEvaluator param for metric name in evaluation (supports "f1" (default), "weightedPrecision", "weightedRecall", "accuracy")
  accuracy = MulticlassClassificationEvaluator(labelCol='target',metricName='accuracy')
  Precision = MulticlassClassificationEvaluator(labelCol='target',metricName='weightedPrecision')
  Recall = MulticlassClassificationEvaluator(labelCol='target',metricName='weightedRecall')
  #BinaryClassificationEvaluator param for metric name in evaluation (supports "areaUnderROC" (default), "areaUnderPR")
  UnderROC = BinaryClassificationEvaluator(labelCol='target',metricName='areaUnderROC')
  UnderPR = BinaryClassificationEvaluator(labelCol='target',metricName='areaUnderPR')
  
  acuracia = accuracy.evaluate(predicao)
  Precision = Precision.evaluate(predicao)
  Recall = Recall.evaluate(predicao)
  roc = UnderROC.evaluate(predicao)
  PR = UnderPR.evaluate(predicao)
  
  return acuracia,Precision,Recall,roc,PR

In [19]:
def criar_spark_dataframe_por_dicionario(dicionario):
  colunas = list(dicionario.keys())
  nlin = len(dicionario[colunas[0]])
  lista_nova = []
  linhas = []
  for i in range(nlin):
    for j in colunas:
      lista_nova.append(dicionario[j][i])
    print(lista_nova)
    linhas.append(tuple(lista_nova))
    lista_nova = []
  return spark.createDataFrame(linhas,colunas)

In [20]:
def df_metricas(modelos, bases, nome_bases = ['treino', 'teste'], nome_modelos = []):
  bases = zip(bases,nome_bases)
  if nome_modelos != []:
    modelos = zip(modelos,nome_modelos)
  dados_modelos_bases = {'base': list(),'modelo': list(), 'acuracia': list(), 
                                'Precision': list(), 'Recall': list(), "underROC": list(), "UnderPR": list()}
  #print("recebeu:")
  #print(f"nome bases = {nome_bases}")
  #print(f"bases = {bases}")
  #print(f"nome modelos = {nome_modelos}")
  #print(f"modelos = {modelos}")
  #print("resposta inicial:")
  #print(dados_modelos_bases)
  for base in bases:
    #print("entrou no primeiro for")
    if nome_modelos != []:
      #print("entrou no primeiro if")
      for modelo in modelos :
        #calcula as métricas
        #print(modelo)
        acuracia,Precision,Recall,roc,PR = metricas(modelo[0],base[0])
        print(f"base: {base[1]}\
              \n\tmodelo: {modelo[1]}\
              \n\tAcuracia:{str(acuracia)}\
              \n\tPrecision: {str(Precision)}\
              \n\tRecall:{str(Recall)}\
              \n\tRoc: {str(roc)}\
              \n\tPR: {str(PR)}")
        #ajusta o nome do modelo para compor o dataframe de resposta
        #nome_modelo = str(modelo).split()[0].split("_")[0]
        dados_modelos_bases['base'].append(base[1])
        dados_modelos_bases['modelo'].append(modelo[1])
        dados_modelos_bases['acuracia'].append(round(acuracia,5))
        dados_modelos_bases['Precision'].append(round(Precision,5))
        dados_modelos_bases['Recall'].append(round(Recall,5))
        dados_modelos_bases['underROC'].append(round(roc,5))
        dados_modelos_bases['UnderPR'].append(round(PR,5))
    else:
      #print("entrou no else")
      for modelo in modelos :
        #print("entrou no for de modelos")
        #calcula as métricas
        acuracia,Precision,Recall,roc,PR = metricas(modelo,base[0])
        #ajusta o nome do modelo para compor o dataframe de resposta
        nome_modelo = str(modelo).split()[0].split("_")[0]
        print(f"base: {base[1]}\
              \n\tmodelo: {nome_modelo}\
              \n\tAcuracia:{str(acuracia)}\
              \n\tPrecision: {str(Precision)}\
              \n\tRecall:{str(Recall)}\
              \n\tRoc: {str(roc)}\
              \n\tPR: {str(PR)}")
        dados_modelos_bases['base'].append(base[1])
        dados_modelos_bases['modelo'].append(nome_modelo)
        dados_modelos_bases['acuracia'].append(round(acuracia,5))
        dados_modelos_bases['Precision'].append(round(Precision,5))
        dados_modelos_bases['Recall'].append(round(Recall,5))
        dados_modelos_bases['underROC'].append(round(roc,5))
        dados_modelos_bases['UnderPR'].append(round(PR,5))
        
  print(dados_modelos_bases)
  return criar_spark_dataframe_por_dicionario(dados_modelos_bases)

In [21]:
Evaluator = MulticlassClassificationEvaluator(labelCol='target')
df_tabelao = spark.read.parquet('/FileStore/olist/ENTREGA_2/0_bases/ABT_com_nulo.parquet')
df_tabelao = df_tabelao.fillna(0)
vetorizar= vetorizar()
df_tabelao = vetorizar.transform(df_tabelao)
treino, teste = df_tabelao.randomSplit([0.7, 0.3], seed=42)
treino = treino.coalesce(2).cache()#.cache().repartition(2)
teste = teste.coalesce(2).cache()#.cache().repartition(2)

####LogisticRegressionModel

In [23]:
crossval_LogisticRegression = CrossValidator(estimator=new_LogisticRegression,
                                             estimatorParamMaps=paramGrid_LogisticRegressionModel,
                                             evaluator=Evaluator,
                                             numFolds=10)
modelo_cross_reg_log = crossval_LogisticRegression.fit(treino)
#25.67 minutes
melhorModelo_reg_log = modelo_cross_reg_log.bestModel
print(melhorModelo_reg_log.intercept)
print(melhorModelo_reg_log.coefficients)
print(melhorModelo_reg_log.getParam)
print ('Best Param (regParam): ', melhorModelo_reg_log._java_obj.getRegParam())
print ('Best Param (MaxIter): ', melhorModelo_reg_log._java_obj.getMaxIter())
print ('Best Param (elasticNetParam): ', melhorModelo_reg_log._java_obj.getElasticNetParam())
melhorModelo_reg_log.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_LogisticRegression')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1756330769975410> in <module> 
 12 print ( 'Best Param (MaxIter): ' , melhorModelo_reg_log . _java_obj . getMaxIter ( ) ) 
 13 print ( 'Best Param (elasticNetParam): ' , melhorModelo_reg_log . _java_obj . getElasticNetParam ( ) ) 
 ---> 14 melhorModelo_reg_log . save ( '/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_LogisticRegression' ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 258 def save ( self , path ) : 
 259 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 260 self . write ( ) . save ( path ) 
 261 
 262 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 197 if not isinstance ( path , basestring ) : 
 198 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 199 self . _jwrite . save ( path ) 
 200 
 201 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o43279.save.
: java.io.IOException: Path /FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_LogisticRegression already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:702)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:179)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [24]:
melhorModelo_LogisticRegression = LogisticRegressionModel.load("/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_LogisticRegression")
print ('Best Param (regParam): ', melhorModelo_LogisticRegression._java_obj.getRegParam())
print ('Best Param (MaxIter): ', melhorModelo_LogisticRegression._java_obj.getMaxIter())
print ('Best Param (elasticNetParam): ', melhorModelo_LogisticRegression._java_obj.getElasticNetParam())

Best Param (regParam): 0.001
Best Param (MaxIter): 10
Best Param (elasticNetParam): 0.8

####DecisionTree

In [26]:
crossval_DecisionTreeClassifier = CrossValidator(estimator=new_DecisionTreeClassifier,
                                                 estimatorParamMaps=paramGrid_DecisionTree,
                                                 evaluator=Evaluator,
                                                 numFolds=10)
modelo_cross_DecisionTreeClassifier = crossval_DecisionTreeClassifier.fit(treino)
#46.24 minutes
melhorModelo_DecisionTreeClassifier = modelo_cross_DecisionTreeClassifier.bestModel
print(melhorModelo_DecisionTreeClassifier.getParam)
print ('Best Param (maxDepth): ', melhorModelo_DecisionTreeClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_DecisionTreeClassifier._java_obj.getMaxBins())
print ('Best Param (minInfoGain): ', melhorModelo_DecisionTreeClassifier._java_obj.getMinInfoGain())
print ('Best Param (impurity): ', melhorModelo_DecisionTreeClassifier._java_obj.getImpurity())
print ('Best Param (seed): ', melhorModelo_DecisionTreeClassifier._java_obj.getSeed())

"""
Best Param (maxDepth):  10
Best Param (maxBins):  10
Best Param (minInfoGain):  0.01
Best Param (impurity):  entropy
Best Param (seed):  956191873026065186
"""
melhorModelo_DecisionTreeClassifier.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_DecisionTreeClassifier')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1756330769975409> in <module> 
 20 Best Param ( seed ) : 956191873026065186 
 21 """
 ---> 22 melhorModelo_DecisionTreeClassifier . save ( '/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_DecisionTreeClassifier' ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 258 def save ( self , path ) : 
 259 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 260 self . write ( ) . save ( path ) 
 261 
 262 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 197 if not isinstance ( path , basestring ) : 
 198 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 199 self . _jwrite . save ( path ) 
 200 
 201 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o64202.save.
: java.io.IOException: Path /FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_DecisionTreeClassifier already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:702)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:179)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [27]:
melhorModelo_DecisionTreeClassifier =DecisionTreeClassificationModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_DecisionTreeClassifier')
print ('Best Param (maxDepth): ', melhorModelo_DecisionTreeClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_DecisionTreeClassifier._java_obj.getMaxBins())
print ('Best Param (minInfoGain): ', melhorModelo_DecisionTreeClassifier._java_obj.getMinInfoGain())
print ('Best Param (impurity): ', melhorModelo_DecisionTreeClassifier._java_obj.getImpurity())

Best Param (maxDepth): 10
Best Param (maxBins): 10
Best Param (minInfoGain): 0.01
Best Param (impurity): entropy

####SVM

In [29]:
crossval_LinearSVC2 = CrossValidator(estimator=new_LinearSVC2,
                                    estimatorParamMaps=paramGrid_LinearSVC2,
                                    evaluator=Evaluator,
                                    numFolds=10)
modelo_cross_LinearSVC2 = crossval_LinearSVC2.fit(treino)
#3.83 hours
melhorModelo_cross_LinearSVC2 = modelo_cross_LinearSVC2.bestModel
print(melhorModelo_cross_LinearSVC2.getParam)
print ('Best Param (regParam): ', melhorModelo_cross_LinearSVC2._java_obj.getRegParam())
print ('Best Param (MaxIter): ', melhorModelo_cross_LinearSVC2._java_obj.getMaxIter())
"""
Best Param (regParam):  0.01
Best Param (MaxIter):  100
Out[423]: '\nBest Param (regParam):  0.0\nBest Param (MaxIter):  100\n'
"""
melhorModelo_cross_LinearSVC2.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_LinearSVC2')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1756330769975408> in <module> 
 14 Out [ 423 ] : '\nBest Param (regParam): 0.0\nBest Param (MaxIter): 100\n' 
 15 """
 ---> 16 melhorModelo_cross_LinearSVC2 . save ( '/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_LinearSVC2' ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 258 def save ( self , path ) : 
 259 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 260 self . write ( ) . save ( path ) 
 261 
 262 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 197 if not isinstance ( path , basestring ) : 
 198 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 199 self . _jwrite . save ( path ) 
 200 
 201 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o12023.save.
: java.io.IOException: Path /FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_LinearSVC2 already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:702)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:179)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [30]:
melhorModelo_cross_LinearSVC = LinearSVCModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_LinearSVC2')
print ('Best Param (regParam): ', melhorModelo_cross_LinearSVC._java_obj.getRegParam())
print ('Best Param (MaxIter): ', melhorModelo_cross_LinearSVC._java_obj.getMaxIter())

Best Param (regParam): 0.01
Best Param (MaxIter): 100

####RandomForestClassifier

In [32]:
# crossval_RandomForestClassifier = CrossValidator(estimator=new_RandomForestClassifier,
#                                                  estimatorParamMaps=paramGrid_RandomForestClassifier,
#                                                  evaluator=Evaluator,
#                                                  numFolds=10)
# modelo_cross_RandomForestClassifier = crossval_RandomForestClassifier.fit(treino)
# melhorModelo_cross_RandomForestClassifier = modelo_cross_RandomForestClassifier.bestModel
# print(melhorModelo_cross_RandomForestClassifier.getParam)
# print ('Best Param (maxDepth): ', melhorModelo_cross_RandomForestClassifier._java_obj.getMaxDepth())
# print ('Best Param (maxBins): ', melhorModelo_cross_RandomForestClassifier._java_obj.getMaxBins())
# print ('Best Param (minInfoGain): ', melhorModelo_cross_RandomForestClassifier._java_obj.getMinInfoGain())
# print ('Best Param (impurity): ', melhorModelo_cross_RandomForestClassifier._java_obj.getImpurity())
# print ('Best Param (seed): ', melhorModelo_cross_RandomForestClassifier._java_obj.getSeed())
# print ('Best Param (numTrees): ', melhorModelo_cross_RandomForestClassifier._java_obj.getNumTrees())
# melhorModelo_cross_RandomForestClassifier.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_RandomForestClassifier')

Como não funcionou o CrossValidator para RandomForestClassifier, então fizemos manualmente alguns modelos com parâmetros diferentes e comparamos os resultados

In [34]:
fit_RandomForestClassifier1 = RandomForestClassifier1.fit(treino)
# fit_RandomForestClassifier2 = RandomForestClassifier2.fit(treino)
# fit_RandomForestClassifier3 = RandomForestClassifier3.fit(treino)
# fit_RandomForestClassifier4 = RandomForestClassifier4.fit(treino)
# fit_RandomForestClassifier5 = RandomForestClassifier5.fit(treino)
# fit_RandomForestClassifier6 = RandomForestClassifier6.fit(treino)
# fit_RandomForestClassifier7 = RandomForestClassifier7.fit(treino)
# fit_RandomForestClassifier8 = RandomForestClassifier8.fit(treino)
# fit_RandomForestClassifier9 = RandomForestClassifier9.fit(treino)
# fit_RandomForestClassifier10 = RandomForestClassifier10.fit(treino)
# fit_RandomForestClassifier11 = RandomForestClassifier11.fit(treino)
fit_RandomForestClassifier12 = RandomForestClassifier12.fit(treino)
# fit_RandomForestClassifier13 = RandomForestClassifier13.fit(treino)
# fit_RandomForestClassifier14 = RandomForestClassifier14.fit(treino)
fit_RandomForestClassifier15 = RandomForestClassifier15.fit(treino)
# fit_RandomForestClassifier16 = RandomForestClassifier16.fit(treino)
fit_RandomForestClassifier17 = RandomForestClassifier17.fit(treino)
fit_RandomForestClassifier18 = RandomForestClassifier18.fit(treino)
#1.23 hours

In [35]:
modelos = [fit_RandomForestClassifier1,
           fit_RandomForestClassifier12,
           fit_RandomForestClassifier15,
           fit_RandomForestClassifier17,
           fit_RandomForestClassifier18
          ]
nomes_modelos = ['fit_RandomForestClassifier1',
                 'fit_RandomForestClassifier12',
                 'fit_RandomForestClassifier15',
                 'fit_RandomForestClassifier17',
                 'fit_RandomForestClassifier18']
bases = [teste]
nome_bases = ['teste']
df_metricas_treino_teste_RandomForestClassifier2 =  df_metricas(modelos, bases, nome_bases = nome_bases, nome_modelos=nomes_modelos)
display(df_metricas_treino_teste_RandomForestClassifier2)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste,fit_RandomForestClassifier1,0.81198,0.82488,0.81198,0.72457,0.55039
teste,fit_RandomForestClassifier12,0.83178,0.81919,0.83178,0.77011,0.59659
teste,fit_RandomForestClassifier15,0.82984,0.81607,0.82984,0.76318,0.5905
teste,fit_RandomForestClassifier17,0.80445,0.82908,0.80445,0.75032,0.59431
teste,fit_RandomForestClassifier18,0.78131,0.61045,0.78131,0.5,0.21869


In [36]:
fit_RandomForestClassifier12.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_RandomForestClassifier')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1756330769975500> in <module> 
 ----> 1 fit_RandomForestClassifier12 . save ( '/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_RandomForestClassifier' ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 258 def save ( self , path ) : 
 259 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 260 self . write ( ) . save ( path ) 
 261 
 262 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 197 if not isinstance ( path , basestring ) : 
 198 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 199 self . _jwrite . save ( path ) 
 200 
 201 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o20093.save.
: java.io.IOException: Path /FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_RandomForestClassifier already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:702)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:179)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [37]:
melhorModelo_RandomForestClassifier = RandomForestClassificationModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_RandomForestClassifier')
print ('Best Param (maxDepth): ', melhorModelo_RandomForestClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_RandomForestClassifier._java_obj.getMaxBins())
print ('Best Param (NumTrees): ', melhorModelo_RandomForestClassifier._java_obj.getNumTrees())

Best Param (maxDepth): 20
Best Param (maxBins): 50
Best Param (NumTrees): 50

####Gradient-boosted tree classifier

In [39]:
# crossval_GBTClassifier1 = CrossValidator(estimator=new_GBTClassifier1,
#                                         estimatorParamMaps=paramGrid_GBTClassifier1,
#                                         evaluator=Evaluator,
#                                         numFolds=10)
# melhorModelo_cross_GBTClassifier1 = modelo_cross_GBTClassifier1.bestModel
# print ('Best Param (maxDepth): ', melhorModelo_cross_GBTClassifier1._java_obj.getMaxDepth())
# print ('Best Param (maxBins): ', melhorModelo_cross_GBTClassifier1._java_obj.getMaxBins())
# print ('Best Param (minInfoGain): ', melhorModelo_cross_GBTClassifier1._java_obj.getMinInfoGain())
# print ('Best Param (maxIter): ', melhorModelo_cross_GBTClassifier1._java_obj.getMaxIter())
# melhorModelo_cross_GBTClassifier1.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_GBTClassifier1')

Como não funcionou o CrossValidator para GBTClassifier, então fizemos manualmente alguns modelos com parâmetros diferentes e comparamos os resultados

In [41]:
fit_modeml_new_GBTClassifier1 = new_GBTClassifier1.fit(treino)
# fit_modeml_new_GBTClassifier2 = new_GBTClassifier2.fit(treino)
# fit_modeml_new_GBTClassifier3 = new_GBTClassifier3.fit(treino)
# fit_modeml_new_GBTClassifier4 = new_GBTClassifier4.fit(treino)
#2.12 minutes
# fit_modeml_new_GBTClassifier5 = new_GBTClassifier5.fit(treino)
# fit_modeml_new_GBTClassifier6 = new_GBTClassifier6.fit(treino)
# fit_modeml_new_GBTClassifier7 = new_GBTClassifier7.fit(treino)
# fit_modeml_new_GBTClassifier8 = new_GBTClassifier8.fit(treino)
# 3.04 minutes
# fit_modeml_new_GBTClassifier9 = new_GBTClassifier9.fit(treino)
# fit_modeml_new_GBTClassifier10 = new_GBTClassifier10.fit(treino)
# fit_modeml_new_GBTClassifier11 = new_GBTClassifier11.fit(treino)
# fit_modeml_new_GBTClassifier12 = new_GBTClassifier12.fit(treino)
#10.13 minutes
# fit_modeml_new_GBTClassifier13 = new_GBTClassifier13.fit(treino)
# fit_modeml_new_GBTClassifier14 = new_GBTClassifier14.fit(treino)
fit_modeml_new_GBTClassifier15 = new_GBTClassifier15.fit(treino)
# fit_modeml_new_GBTClassifier16 = new_GBTClassifier16.fit(treino)
#3.30 minutes
# fit_modeml_new_GBTClassifier17 = new_GBTClassifier17.fit(treino)
# fit_modeml_new_GBTClassifier18 = new_GBTClassifier18.fit(treino)
# fit_modeml_new_GBTClassifier19 = new_GBTClassifier19.fit(treino)
# fit_modeml_new_GBTClassifier20 = new_GBTClassifier20.fit(treino)
#3.92 minutes
# fit_modeml_new_GBTClassifier21 = new_GBTClassifier21.fit(treino)
# fit_modeml_new_GBTClassifier22 = new_GBTClassifier22.fit(treino)
# fit_modeml_new_GBTClassifier23 = new_GBTClassifier23.fit(treino)
# fit_modeml_new_GBTClassifier24 = new_GBTClassifier24.fit(treino)
# #9.18 minutes
# fit_modeml_new_GBTClassifier25 = new_GBTClassifier25.fit(treino)
# fit_modeml_new_GBTClassifier26 = new_GBTClassifier26.fit(treino)
# fit_modeml_new_GBTClassifier27 = new_GBTClassifier27.fit(treino)
# fit_modeml_new_GBTClassifier28 = new_GBTClassifier28.fit(treino)
# #51.45 minutes
# fit_modeml_new_GBTClassifier29 = new_GBTClassifier29.fit(treino)
# fit_modeml_new_GBTClassifier30 = new_GBTClassifier30.fit(treino)
# fit_modeml_new_GBTClassifier31 = new_GBTClassifier31.fit(treino)
# fit_modeml_new_GBTClassifier32 = new_GBTClassifier32.fit(treino)
# #23.11 minutes
# fit_modeml_new_GBTClassifier33 = new_GBTClassifier33.fit(treino)
# fit_modeml_new_GBTClassifier34 = new_GBTClassifier34.fit(treino)
fit_modeml_new_GBTClassifier35 = new_GBTClassifier35.fit(treino)
# fit_modeml_new_GBTClassifier36 = new_GBTClassifier36.fit(treino)
#54.54 minutes

fit_modeml_new_GBTClassifier37 = new_GBTClassifier37.fit(treino)
# fit_modeml_new_GBTClassifier38 = new_GBTClassifier38.fit(treino)
# fit_modeml_new_GBTClassifier39 = new_GBTClassifier39.fit(treino)
# fit_modeml_new_GBTClassifier40 = new_GBTClassifier40.fit(treino)

# fit_modeml_new_GBTClassifier41 = new_GBTClassifier41.fit(treino)
# fit_modeml_new_GBTClassifier42 = new_GBTClassifier42.fit(treino)
fit_modeml_new_GBTClassifier43 = new_GBTClassifier43.fit(treino)
# #4.46 hours

In [42]:
modelos = [fit_modeml_new_GBTClassifier1,
#            fit_modeml_new_GBTClassifier2,
#            fit_modeml_new_GBTClassifier3,
#            fit_modeml_new_GBTClassifier4,
#            fit_modeml_new_GBTClassifier5,
#            fit_modeml_new_GBTClassifier6,
#            fit_modeml_new_GBTClassifier7,
#            fit_modeml_new_GBTClassifier8,
#            fit_modeml_new_GBTClassifier9,
#            fit_modeml_new_GBTClassifier10,
#            fit_modeml_new_GBTClassifier11,
#            fit_modeml_new_GBTClassifier12,
#            fit_modeml_new_GBTClassifier13,
#            fit_modeml_new_GBTClassifier14,
           fit_modeml_new_GBTClassifier15,
#            fit_modeml_new_GBTClassifier16,
#            fit_modeml_new_GBTClassifier17,
#            fit_modeml_new_GBTClassifier18,
#            fit_modeml_new_GBTClassifier19,
#            fit_modeml_new_GBTClassifier20,
#            fit_modeml_new_GBTClassifier21,
#            fit_modeml_new_GBTClassifier22,
#            fit_modeml_new_GBTClassifier23,
#            fit_modeml_new_GBTClassifier24,
#            fit_modeml_new_GBTClassifier25,
#            fit_modeml_new_GBTClassifier26,
#            fit_modeml_new_GBTClassifier27,
#            fit_modeml_new_GBTClassifier28,
#            fit_modeml_new_GBTClassifier29,
#            fit_modeml_new_GBTClassifier30,
#            fit_modeml_new_GBTClassifier31,
#            fit_modeml_new_GBTClassifier32,
#            fit_modeml_new_GBTClassifier33,
#            fit_modeml_new_GBTClassifier34,
           fit_modeml_new_GBTClassifier35,
#            fit_modeml_new_GBTClassifier36,
           fit_modeml_new_GBTClassifier37,
#            fit_modeml_new_GBTClassifier38,
#            fit_modeml_new_GBTClassifier39,
#            fit_modeml_new_GBTClassifier40,
#            fit_modeml_new_GBTClassifier41,
#            fit_modeml_new_GBTClassifier42,
           fit_modeml_new_GBTClassifier43]
nomes_modelos = ['GBTClassifier1',
#                 'GBTClassifier2',
#                 'GBTClassifier3',
#                 'GBTClassifier4',
#                 'GBTClassifier5',
#                 'GBTClassifier6',
#                 'GBTClassifier7',
#                 'GBTClassifier8',
#                 'GBTClassifier9',
#                 'GBTClassifier10',
#                 'GBTClassifier11',
#                 'GBTClassifier12',
#                 'GBTClassifier13',
#                 'GBTClassifier14',
                'GBTClassifier15',
#                 'GBTClassifier16',
#                 'GBTClassifier17',
#                 'GBTClassifier18',
#                 'GBTClassifier19',
#                 'GBTClassifier20',
#                 'GBTClassifier21',
#                 'GBTClassifier22',
#                 'GBTClassifier23',
#                 'GBTClassifier24',
#                 'GBTClassifier25',
#                 'GBTClassifier26',
#                 'GBTClassifier27',
#                 'GBTClassifier28',
#                 'GBTClassifier29',
#                 'GBTClassifier30',
#                 'GBTClassifier31',
#                 'GBTClassifier32',
#                 'GBTClassifier33',
#                 'GBTClassifier34',
                'GBTClassifier35',
#                 'GBTClassifier36',
                'GBTClassifier37',
#                 'GBTClassifier38',
#                 'GBTClassifier39',
#                 'GBTClassifier40',
#                 'GBTClassifier41',
#                 'GBTClassifier42',
                'GBTClassifier43']
bases = [teste]#[treino,teste]
nome_bases = ["teste"]#['treino','teste']
df_metricas_treino_teste_GBTClassifier =  df_metricas(modelos, bases, nome_bases = nome_bases, nome_modelos=nomes_modelos)

base: teste 
	modelo: GBTClassifier1 
	Acuracia:0.8271879483500717 
	Precision: 0.8122946733694704 
	Recall:0.8271879483500717 
	Roc: 0.739186473410615 
	PR: 0.5792750932794747
base: teste 
	modelo: GBTClassifier15 
	Acuracia:0.8301649928263989 
	Precision: 0.8199548420002909 
	Recall:0.8301649928263988 
	Roc: 0.7445541143666685 
	PR: 0.5961057520435517
base: teste 
	modelo: GBTClassifier35 
	Acuracia:0.8295552367288379 
	Precision: 0.8190873246318453 
	Recall:0.8295552367288379 
	Roc: 0.7397787212592131 
	PR: 0.5768969061809907
base: teste 
	modelo: GBTClassifier37 
	Acuracia:0.8311334289813487 
	Precision: 0.8195309715477694 
	Recall:0.8311334289813487 
	Roc: 0.7473141427923367 
	PR: 0.5822761289127654
base: teste 
	modelo: GBTClassifier43 
	Acuracia:0.8095050215208034 
	Precision: 0.7891307527330329 
	Recall:0.8095050215208034 
	Roc: 0.6698438519576605 
	PR: 0.5042395970123179
{'base': ['teste', 'teste', 'teste', 'teste', 'teste'], 'modelo': ['GBTClassifier1', 'GBTClassifier15', 'GBTClassifier35', 'GBTClassifier37', 'GBTClassifier43'], 'acuracia': [0.82719, 0.83016, 0.82956, 0.83113, 0.80951], 'Precision': [0.81229, 0.81995, 0.81909, 0.81953, 0.78913], 'Recall': [0.82719, 0.83016, 0.82956, 0.83113, 0.80951], 'underROC': [0.73919, 0.74455, 0.73978, 0.74731, 0.66984], 'UnderPR': [0.57928, 0.59611, 0.5769, 0.58228, 0.50424]}
['teste', 'GBTClassifier1', 0.82719, 0.81229, 0.82719, 0.73919, 0.57928]
['teste', 'GBTClassifier15', 0.83016, 0.81995, 0.83016, 0.74455, 0.59611]
['teste', 'GBTClassifier35', 0.82956, 0.81909, 0.82956, 0.73978, 0.5769]
['teste', 'GBTClassifier37', 0.83113, 0.81953, 0.83113, 0.74731, 0.58228]
['teste', 'GBTClassifier43', 0.80951, 0.78913, 0.80951, 0.66984, 0.50424]

In [43]:
display(df_metricas_treino_teste_GBTClassifier)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste,GBTClassifier1,0.82719,0.81229,0.82719,0.73919,0.57928
teste,GBTClassifier15,0.83016,0.81995,0.83016,0.74455,0.59611
teste,GBTClassifier35,0.82956,0.81909,0.82956,0.73978,0.5769
teste,GBTClassifier37,0.83113,0.81953,0.83113,0.74731,0.58228
teste,GBTClassifier43,0.80951,0.78913,0.80951,0.66984,0.50424


In [44]:
dbutils.fs.rm('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_GBTClassifier1',True)

Out[48]: True

In [45]:
fit_modeml_new_GBTClassifier37.save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_GBTClassifier1')

In [46]:
melhorModelo_cross_GBTClassifier = GBTClassificationModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_GBTClassifier1')
print ('Best Param (maxDepth): ', melhorModelo_cross_GBTClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_cross_GBTClassifier._java_obj.getMaxBins())
print ('Best Param (minInfoGain): ', melhorModelo_cross_GBTClassifier._java_obj.getMinInfoGain())
print ('Best Param (maxIter): ', melhorModelo_cross_GBTClassifier._java_obj.getMaxIter())

"""
1a vez:
Best Param (maxDepth):  30
Best Param (maxBins):  20
Best Param (minInfoGain):  0.01
Best Param (maxIter):  100

2a vez:
Best Param (maxDepth):  30
Best Param (maxBins):  50
Best Param (minInfoGain):  0.01
Best Param (maxIter):  100
"""

Best Param (maxDepth): 30
Best Param (maxBins): 50
Best Param (minInfoGain): 0.01
Best Param (maxIter): 100
Out[50]: '\n1a vez:\nBest Param (maxDepth): 30\nBest Param (maxBins): 20\nBest Param (minInfoGain): 0.01\nBest Param (maxIter): 100\n\n'

##5.7.3 Avaliação dos modelos

In [48]:
def df_metricas(modelos, bases, nome_bases = ['treino', 'teste']):
  bases = zip(bases,nome_bases)
  dados_modelos_bases = {'base': list(),'modelo': list(), 'acuracia': list(), 
                                'Precision': list(), 'Recall': list(), "underROC": list(), "UnderPR": list()}
  for base in bases:
    for modelo in modelos :
      #calcula as métricas
      acuracia,Precision,Recall,roc,PR = metricas(modelo,base[0])
      #ajusta o nome do modelo para compor o dataframe de resposta
      nome_modelo = str(modelo).split()[0].split("_")[0]
      dados_modelos_bases['base'].append(base[1])
      dados_modelos_bases['modelo'].append(nome_modelo)
      dados_modelos_bases['acuracia'].append(round(acuracia,5))
      dados_modelos_bases['Precision'].append(round(Precision,5))
      dados_modelos_bases['Recall'].append(round(Recall,5))
      dados_modelos_bases['underROC'].append(round(roc,5))
      dados_modelos_bases['UnderPR'].append(round(PR,5))
  return criar_spark_dataframe_por_dicionario(dados_modelos_bases)

In [49]:
#define os modelos que vão ser avaliads
modelos = [melhorModelo_LogisticRegression,
           melhorModelo_DecisionTreeClassifier,
           melhorModelo_cross_LinearSVC,
           melhorModelo_RandomForestClassifier,
           melhorModelo_cross_GBTClassifier]
n_modelos=['melhorModelo_LogisticRegression',
           'melhorModelo_DecisionTreeClassifier',
           'melhorModelo_cross_LinearSVC',
           'melhorModelo_RandomForestClassifier',
           'melhorModelo_cross_GBTClassifier']

#define as bases que serão avaliadas
bases = [treino,teste]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_hiperparametros', 'teste_hiperparametros']
# df_metricas_primeiro =  df_metricas(modelos, bases, nome_bases)
df_metricas_treino_teste_hiperparametros =  df_metricas(modelos, bases, nome_bases = nome_bases)

base: treino_hiperparametros 
	modelo: LogisticRegressionModel: 
	Acuracia:0.8307414988848493 
	Precision: 0.8172088490614913 
	Recall:0.8307414988848492 
	Roc: 0.7627776829713615 
	PR: 0.5834358484290693
base: treino_hiperparametros 
	modelo: DecisionTreeClassificationModel 
	Acuracia:0.8328495921297853 
	Precision: 0.8191463205359183 
	Recall:0.8328495921297853 
	Roc: 0.31035661106666756 
	PR: 0.1601372540577856
base: treino_hiperparametros 
	modelo: LinearSVC 
	Acuracia:0.8152821484219853 
	Precision: 0.8031924495837579 
	Recall:0.8152821484219853 
	Roc: 0.7276853081472351 
	PR: 0.5281023663452102
base: treino_hiperparametros 
	modelo: RandomForestClassificationModel 
	Acuracia:0.8647765115639607 
	Precision: 0.8641879728521549 
	Recall:0.8647765115639607 
	Roc: 0.8347843434041008 
	PR: 0.7203955146151719
base: treino_hiperparametros 
	modelo: GBTClassificationModel 
	Acuracia:0.8456050838654486 
	Precision: 0.8405209473221958 
	Recall:0.8456050838654486 
	Roc: 0.7647207263268099 
	PR: 0.6408864331884253
base: teste_hiperparametros 
	modelo: LogisticRegressionModel: 
	Acuracia:0.826793400286944 
	Precision: 0.8118800991822174 
	Recall:0.826793400286944 
	Roc: 0.7608317856258761 
	PR: 0.5803066547421976
base: teste_hiperparametros 
	modelo: DecisionTreeClassificationModel 
	Acuracia:0.8279053084648493 
	Precision: 0.8130241932819376 
	Recall:0.8279053084648493 
	Roc: 0.3165502218262107 
	PR: 0.16200538886428173
base: teste_hiperparametros 
	modelo: LinearSVC 
	Acuracia:0.8134146341463414 
	Precision: 0.8012062585788443 
	Recall:0.8134146341463415 
	Roc: 0.7221009997887958 
	PR: 0.5166359807608738
base: teste_hiperparametros 
	modelo: RandomForestClassificationModel 
	Acuracia:0.8317790530846485 
	Precision: 0.8191877332743193 
	Recall:0.8317790530846485 
	Roc: 0.7701055728454258 
	PR: 0.5965921853389956
base: teste_hiperparametros 
	modelo: GBTClassificationModel 
	Acuracia:0.8311334289813487 
	Precision: 0.8195309715477694 
	Recall:0.8311334289813487 
	Roc: 0.7473141427923365 
	PR: 0.5822761289127653
{'base': ['treino_hiperparametros', 'treino_hiperparametros', 'treino_hiperparametros', 'treino_hiperparametros', 'treino_hiperparametros', 'teste_hiperparametros', 'teste_hiperparametros', 'teste_hiperparametros', 'teste_hiperparametros', 'teste_hiperparametros'], 'modelo': ['LogisticRegressionModel:', 'DecisionTreeClassificationModel', 'LinearSVC', 'RandomForestClassificationModel', 'GBTClassificationModel', 'LogisticRegressionModel:', 'DecisionTreeClassificationModel', 'LinearSVC', 'RandomForestClassificationModel', 'GBTClassificationModel'], 'acuracia': [0.83074, 0.83285, 0.81528, 0.86478, 0.84561, 0.82679, 0.82791, 0.81341, 0.83178, 0.83113], 'Precision': [0.81721, 0.81915, 0.80319, 0.86419, 0.84052, 0.81188, 0.81302, 0.80121, 0.81919, 0.81953], 'Recall': [0.83074, 0.83285, 0.81528, 0.86478, 0.84561, 0.82679, 0.82791, 0.81341, 0.83178, 0.83113], 'underROC': [0.76278, 0.31036, 0.72769, 0.83478, 0.76472, 0.76083, 0.31655, 0.7221, 0.77011, 0.74731], 'UnderPR': [0.58344, 0.16014, 0.5281, 0.7204, 0.64089, 0.58031, 0.16201, 0.51664, 0.59659, 0.58228]}
['treino_hiperparametros', 'LogisticRegressionModel:', 0.83074, 0.81721, 0.83074, 0.76278, 0.58344]
['treino_hiperparametros', 'DecisionTreeClassificationModel', 0.83285, 0.81915, 0.83285, 0.31036, 0.16014]
['treino_hiperparametros', 'LinearSVC', 0.81528, 0.80319, 0.81528, 0.72769, 0.5281]
['treino_hiperparametros', 'RandomForestClassificationModel', 0.86478, 0.86419, 0.86478, 0.83478, 0.7204]
['treino_hiperparametros', 'GBTClassificationModel', 0.84561, 0.84052, 0.84561, 0.76472, 0.64089]
['teste_hiperparametros', 'LogisticRegressionModel:', 0.82679, 0.81188, 0.82679, 0.76083, 0.58031]
['teste_hiperparametros', 'DecisionTreeClassificationModel', 0.82791, 0.81302, 0.82791, 0.31655, 0.16201]
['teste_hiperparametros', 'LinearSVC', 0.81341, 0.80121, 0.81341, 0.7221, 0.51664]
['teste_hiperparametros', 'RandomForestClassificationModel', 0.83178, 0.81919, 0.83178, 0.77011, 0.59659]
['tes

In [50]:
display(df_metricas_treino_teste_hiperparametros)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_hiperparametros,LogisticRegressionModel:,0.83074,0.81721,0.83074,0.76278,0.58344
treino_hiperparametros,DecisionTreeClassificationModel,0.83285,0.81915,0.83285,0.31036,0.16014
treino_hiperparametros,LinearSVC,0.81528,0.80319,0.81528,0.72769,0.5281
treino_hiperparametros,RandomForestClassificationModel,0.86478,0.86419,0.86478,0.83478,0.7204
treino_hiperparametros,GBTClassificationModel,0.84561,0.84052,0.84561,0.76472,0.64089
teste_hiperparametros,LogisticRegressionModel:,0.82679,0.81188,0.82679,0.76083,0.58031
teste_hiperparametros,DecisionTreeClassificationModel,0.82791,0.81302,0.82791,0.31655,0.16201
teste_hiperparametros,LinearSVC,0.81341,0.80121,0.81341,0.7221,0.51664
teste_hiperparametros,RandomForestClassificationModel,0.83178,0.81919,0.83178,0.77011,0.59659
teste_hiperparametros,GBTClassificationModel,0.83113,0.81953,0.83113,0.74731,0.58228


In [51]:
df_metricas_treino_teste_hiperparametros.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/Comparacao_metricas_treino_teste_hiperparametros.csv')

Se olharmos somente esta etapa 5.7, os melhores modelos seriam GBTClassifier e RandomForestClassifier